In [1]:
import numpy as np
import pandas as pd
import streamlit as st
from datetime import datetime, timedelta
import os
import gspread
from google.oauth2 import service_account
from gspread_dataframe import set_with_dataframe
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from configs import *
import requests
from functionssb import *

In [2]:
gs_connection = connect_to_gs(st.secrets["gcp_service_account"])

In [3]:
drinks = fetch_drinks_data(gs_connection, drinks_table, prod_google_sheet_key, ['event','drink_size', 'start_time', 'end_time'])

In [4]:
drinks

,event,nominator_name,drinker_name,drink_type,nomination_created_date,nomination_deadline_date,nomination_completed_date,quantity,drink_size,start_time,end_time
0,1,auto,Jonno Frame,last place,15/08/23 05:39:44,22/08/23 23:59,22/08/23 19:29,,330.0,4.25,12.04
1,1,Devlin Sharp,Connor McDonald,nomination,15/08/23 08:45:50,22/08/23 23:59,15/08/23 16:46,,330.0,5.38,11.04
2,1,Devlin Sharp,Tijil Jugadoe,nomination,15/08/23 08:45:50,22/08/23 23:59,17/08/23 18:26,,330.0,0.00,10.88
3,1,Devlin Sharp,Niklas Wietzorrek,nomination,15/08/23 08:45:50,22/08/23 23:59,19/08/23 18:30,,330.0,1.03,7.38
4,2,auto,Tijil Jugadoe,red card,22/08/23 11:02:45,29/08/23 23:59,22/08/23 09:07,,440.0,0.00,8.70
5,2,auto,Rhys Floyd,red card,22/08/23 11:02:45,29/08/23 23:59,Not Completed,,NaN,NaN,NaN
6,2,auto,Lawrence Fowle,own goal,22/08/23 11:02:45,29/08/23 23:59,25/08/23 12:51,,425.0,4.82,10.16
7,2,auto,Divyam Dixit,missed pen,22/08/23 11:02:45,29/08/23 23:59,29/08/23 16:29,,330.0,NaN,NaN
8,2,auto,Alex Wietzorrek,missed pen,22/08/23 11:02:45,29/08/23 23:59,23/08/23 18:01,,330.0,2.20,6.40
9,2,auto,Jason Perry,missed pen,22/08/23 11:02:45,29/08/23 23:59,22/08/23 09:11,,330.0,1.90,7.12


In [5]:
df = categories(drinks)
df

/Users/connormcdonald/Desktop/GitHub/fpl-v2/functionssb.py:507: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"drinker_name": "Name"}, inplace=True)
/Users/connormcdonald/Desktop/GitHub/fpl-v2/functionssb.py:510: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["nomination_completed_date"] = pd.to_datetime(df["nomination_completed_date"], errors="coerce")
/Users/connormcdonald/Desktop/GitHub/fpl-v2/functionssb.py:511: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

,Name,nomination_deadline_date,nomination_completed_date,quantity,Category
0,Jonno Frame,2023-08-22 23:59:00,2023-08-22 19:29:00,1,Completed
1,Connor McDonald,2023-08-22 23:59:00,2023-08-15 16:46:00,1,Completed
2,Tijil Jugadoe,2023-08-22 23:59:00,2023-08-17 18:26:00,1,Completed
3,Niklas Wietzorrek,2023-08-22 23:59:00,2023-08-19 18:30:00,1,Completed
4,Tijil Jugadoe,2023-08-29 23:59:00,2023-08-22 09:07:00,1,Completed
5,Rhys Floyd,2023-08-29 23:59:00,NaT,1,Outstanding
6,Lawrence Fowle,2023-08-29 23:59:00,2023-08-25 12:51:00,1,Completed
7,Divyam Dixit,2023-08-29 23:59:00,2023-08-29 16:29:00,1,Completed
8,Alex Wietzorrek,2023-08-29 23:59:00,2023-08-23 18:01:00,1,Completed
9,Jason Perry,2023-08-29 23:59:00,2023-08-22 09:11:00,1,Completed


In [6]:
df.dtypes

Name                                 object
nomination_deadline_date     datetime64[ns]
nomination_completed_date    datetime64[ns]
quantity                              int64
Category                             object
dtype: object

In [7]:
a = df.iloc[33]["nomination_deadline_date"]

In [8]:
b = df.iloc[33]["nomination_completed_date"]

In [9]:
a>b

True